In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dense,Flatten,Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard
import matplotlib.pyplot as plt
from skimage import color,exposure
import os 
import datetime
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Conv1D
from hyperopt import hp,STATUS_OK,tpe,Trials,fmin
%load_ext tensorboard



The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd data


C:\Users\benia\Desktop\Machine_Learning\Prediction road sign by computer vision\data


In [24]:
train = pd.read_pickle("train.p")
test = pd.read_pickle("test.p")

In [25]:
X_train , y_train = train["features"],train["labels"]
X_test , y_test = test["features"],test["labels"]

X_train.shape,y_train.shape , X_test.shape,y_test.shape

((34799, 32, 32, 3), (34799,), (4410, 32, 32, 3), (4410,))

In [26]:
df = pd.read_csv("signnames.csv")

In [27]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [28]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [50]:
def train_model(model,X_train,y_train,params_fit={}):
    model.compile(loss="categorical_crossentropy",optimizer="Adam",metrics=["accuracy"])
    
    logdir = os.path.join("logs",datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir,histogram_freq=1)
    
    model.fit(X_train,
              y_train,
              batch_size = params_fit.get("batch_size",128),
              epochs = params_fit.get("epochs",5),
              verbose = params_fit.get("verbose",1),
              validation_data = params_fit.get("validation_data",(X_train,y_train)),
              callbacks = [tensorboard_callback]      
             )
    
    return model

In [51]:
def predict (model_trained, X_test,y_test,scoring=accuracy_score):

    y_test_norm = np.argmax(y_test,axis=1)
    
    y_pred_prob = model_trained.predict(X_test)
    y_pred = np.argmax(y_pred_prob,axis=1)

    return scoring(y_test_norm,y_pred)

In [55]:
def get_model(params):
    return Sequential([
    Conv2D(filters=32,kernel_size=(3,3),activation="relu",input_shape = input_shape),
    Conv2D(filters=32,kernel_size=(3,3),activation="relu",padding = "same"),
    MaxPool2D(),
    Dropout(params["droput_cnn_block_one"]),
    
    Conv2D(filters=64,kernel_size=(3,3),activation="relu",padding = "same"),
    Conv2D(filters=64,kernel_size=(3,3),activation="relu"),
    MaxPool2D(),
    Dropout(0.3),
    Dropout(params["droput_cnn_block_two"]),
        
    Conv2D(filters=128,kernel_size=(3,3),activation="relu",padding = "same"),
    Conv2D(filters=128,kernel_size=(3,3),activation="relu"),
    MaxPool2D(),
    Dropout(0.3),
    Dropout(params["droput_cnn_block_three"]),
        
    Flatten(),
    
    Dense(1024,activation="relu"),
    Dropout(0.3),
    Dropout(params["droput_dense_block_one"]),
        
    Dense(1024,activation="relu"),
    Dropout(0.3),
    Dropout(params["droput_dense_block_two"]),
        
    Dense(num_classes, activation="softmax"),
    
    ])




In [44]:
params_fit={"epochs":2}

In [53]:
model = get_cnn_v5(input_shape,num_classes)


In [54]:
model_trained = train_model(model,X_train,y_train)


Train on 34799 samples, validate on 34799 samples
Epoch 1/5
34799/34799 [==============================] - 202s 6ms/sample - loss: 2.5222 - accuracy: 0.2894 - val_loss: 1.1137 - val_accuracy: 0.6486
Epoch 2/5
34799/34799 [==============================] - 237s 7ms/sample - loss: 0.8367 - accuracy: 0.7310 - val_loss: 0.2277 - val_accuracy: 0.9300
Epoch 3/5
34799/34799 [==============================] - 241s 7ms/sample - loss: 0.3769 - accuracy: 0.8841 - val_loss: 0.1041 - val_accuracy: 0.9713
Epoch 4/5
34799/34799 [==============================] - 219s 6ms/sample - loss: 0.2404 - accuracy: 0.9277 - val_loss: 0.0550 - val_accuracy: 0.9847
Epoch 5/5
12288/34799 [=========>....................] - ETA: 2:13 - loss: 0.1990 - accuracy: 0.9419

KeyboardInterrupt: 

In [ ]:
predict(model_trained,X_test,y_test)

In [76]:
def func_obj(params):
    model = get_model(params)
    model.compile(loss="categorical_crossentropy",optimizer="Adam",metrics=["accuracy"])
    
    logdir = os.path.join("logs",datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir,histogram_freq=1)
    
    
    model.fit(
        X_train,
        y_train,
        batch_size = int(params.get("batch_size",128)),
        epochs = 1,
        verbose=0
        )
    accuracy = model.evaluate(X_test,y_test,verbose=0)[1]
    accuracy = score[1]
    print(params,"accuracy={}".format(accuracy))
    
    return{"loss": -accuracy, "status":STATUS_OK , "model":model }

In [77]:
space={
    "batch_size": hp.quniform("batch_size",100,200,10),
    "droput_cnn_block_one" : hp.uniform("droput_cnn_block_one",0.3,0.5),
    "droput_cnn_block_two" : hp.uniform("droput_cnn_block_two",0.3,0.5),
    "droput_cnn_block_three" : hp.uniform("droput_cnn_block_three",0.3,0.5),
    
     "droput_dense_block_one" : hp.uniform("droput_dense_block_one",0.3,0.5),
     "droput_dense_block_two" : hp.uniform("droput_dense_block_two",0.3,0.5),
    
    
}



In [79]:
best = fmin(
    func_obj,
    space,
    tpe.suggest,
    1,
    Trials()
)

  0%|                                                                                                              | 0/1 [02:59<?, ?trial/s, best loss=?]


KeyboardInterrupt: 